In [42]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
import nevergrad as ng
from estival.wrappers.nevergrad import optimize_model
from summer2.utils import ref_times_to_dti


pd.options.plotting.backend =  "plotly"

In [43]:
from summer2 import CompartmentalModel, Stratification, Multiply, AgeStratification
from summer2.parameters import Parameter, Data
from summer2.functions import get_piecewise_scalar_function

**Model 3: SEIQR (Stratified Age + Infectious Compartment)**

In [44]:
COVID_BASE_DATE = datetime(2021,7,1)

In [45]:
def build_model():
    m = CompartmentalModel([0,150], ["S","E","I","Q","R"],"I", ref_date=datetime(2021,7,1))
    m.set_initial_population({"S":   13966223.0 , "E": 3919.0 , "I": 5053.0, "Q": 7255.0, "R": 520973.0})
    m.add_infection_frequency_flow("infection", get_piecewise_scalar_function([7,14,21,28,35,42], [0.6811*Parameter("rate"),0.706*Parameter("rate"),0.6831*Parameter("rate"),0.6205*Parameter("rate"),0.7082*Parameter("rate"),0.5365*Parameter("rate"),0.5331*Parameter("rate")]),"S","E") 
    m.add_transition_flow("progression", 0.2,"E","I")
    m.add_transition_flow("notification", Parameter("detection_rate"),"I","Q")
    m.add_transition_flow("recovery_from_Q", 0.0714,"Q","R") 
    m.add_transition_flow("recovery__from_I", 0.0714,"I","R") 
    #m.add_death_flow("infection_death", Parameter("death_rate"), "I")
    m.request_output_for_flow("infection", "infection")
    m.request_output_for_flow("notification", "notification")
    #m.request_output_for_flow("infection_death", "infection_death")
    #m.request_cumulative_output(name="deaths_cumulative", source="infection_death")
    m.request_output_for_compartments(name="active", compartments=["Q"])

    return m



In [46]:
#Severity Stratification

strata = ["asymptomatic", "symptomatic"]
strat = Stratification(name="severity", strata=strata, compartments=['I'])
strat.set_population_split({"asymptomatic": 0.18, "symptomatic": 0.82})  #set initial values for splitting proportion

strat.set_flow_adjustments("progression", {
    "asymptomatic": None,
    "symptomatic": Multiply(Parameter("symp_prop")),
    })   #from E to Ia and Is


strat.set_flow_adjustments("notification", {
    "asymptomatic": Multiply(Parameter("asymp_det")),
    "symptomatic": None,
    }) #from Ia and Is to Q

In [47]:
#Age Stratification

#strat.set_flow_adjustments("infection_death", {
#"asymptomatic": Multiply(Parameter("asymp_death")),
#"symptomatic": None, })   #adjustment to death rates from Ia and Is

strata_age = [str(n) for n in range(0,80,5)]
strat_age = Stratification(name="age", strata=strata_age, compartments=["S", "E", "I", "Q", "R"])

strat_age.set_population_split({'0': .104, '5': .106, '10': .104, '15': .097, '20': .092 , '25': .085, '30': .076  , '35':  .067 , '40': .06 ,  '45': .052, '50': 0.045,'55': 0.037  , '60':  .03  , '65': .02    ,  '70':  .012   , '75':  .013 })
mixing_matrix = pd.read_pickle("PHL_matrices.pkl", compression='infer')
age_mixing_matrix = mixing_matrix["all_locations"]

strat_age.set_mixing_matrix(age_mixing_matrix)

#question, I'm only using "Stratification" (which is a general stratification command) here, rather than "AgeStratification". Does the code here already indicate that I'm introducing a mixing matrix is already introduced into the transmission dynamics between different age groups?


In [48]:
#Vaccine Stratification

strata = ["unvaccinated", "vaccinated1", "vaccinated2"]
strat_vax = Stratification(name="vax_status", strata=strata, compartments=["S", "E", "I", "Q", "R"])
strat_vax.set_population_split({"unvaccinated": 0.66, "vaccinated1": 0.25, "vaccinated2":0.09})  #set initial values for splitting proportion

strat_vax.set_flow_adjustments("infection", {
    "unvaccinated": None,
    "vaccinated1": Multiply(1. - Parameter("vax_effectiveness1")),
    "vaccinated2": Multiply(1. - Parameter("vax_effectiveness2")) })
#from S to E

In [49]:
#m, strat, strat_age = build_model()
m = build_model()
m.stratify_with(strat)
m.stratify_with(strat_age)
m.stratify_with(strat_vax)


C:\Users\timot\AppData\Local\Temp\ipykernel_7772\1592348534.py:4: DeprecationWarning:

This method is deprecated and scheduled for removal, use get_piecewise_function instead



In [50]:
#Introduce data on vaccination coverage 
vacc1_coverage = [0.2528225419, 0.3336019338, 0.5587344553, 0.6037653232, 0.6727430888, 0.748531439, 0.7896053214]
vacc2_coverage = [0.08541801173, 0.2443839684, 0.3179753753, 0.5131332624, 0.6183547979, 0.6703555428, 0.7474629325]
coverage_start_time = 0
coverage_end_time = 210
coverage_times = range(coverage_start_time, coverage_end_time, 30)


In [51]:
pd.Series(
    vacc1_coverage, 
    index=ref_times_to_dti(COVID_BASE_DATE, coverage_times)
).plot.area(title="Vaccination coverage over time (partial)")


c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [52]:
pd.Series(
    vacc2_coverage, 
    index=ref_times_to_dti(COVID_BASE_DATE, coverage_times)
).plot.area(title="Vaccination coverage over time (full)")

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [53]:
def get_prop_of_remaining_covered(old_prop, new_prop):
    return (new_prop - old_prop) / (1. - old_prop)

interval_prop_unvacc_vaccinated1 = [
    get_prop_of_remaining_covered(
        vacc1_coverage[i],
        vacc1_coverage[i + 1],
    ) 
    for i in range(len(vacc1_coverage) - 1)
]

interval_prop_unvacc_vaccinated2 = [
    get_prop_of_remaining_covered(
        vacc2_coverage[i],
        vacc2_coverage[i + 1],
    ) 
    for i in range(len(vacc1_coverage) - 1)
]

interval_lengths = [
    coverage_times[i + 1] - coverage_times[i] for 
    i in range(len(coverage_times) - 1)
]

In [54]:
pd.Series(
    interval_prop_unvacc_vaccinated1,
    index=ref_times_to_dti(COVID_BASE_DATE, coverage_times[1:]),
).plot(
    kind="scatter",
    title="Proportion of unvaccinated vaccinated during interval (partial)",
)

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [55]:
pd.Series(
    interval_prop_unvacc_vaccinated2,
    index=ref_times_to_dti(COVID_BASE_DATE, coverage_times[1:]),
).plot(
    kind="scatter",
    title="Proportion of unvaccinated vaccinated during interval (full)",
)

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [56]:
def get_rate_from_coverage_and_duration(coverage_increment: float, duration: float) -> float:
    assert duration >= 0.0, f"Duration request is negative: {duration}"
    assert 0.0 <= coverage_increment <= 1.0, f"Coverage increment not in [0, 1]: {coverage_increase}"
    return -np.log(1.0 - coverage_increment) / duration

vaccination_rates1 = [
    get_rate_from_coverage_and_duration(i, j) for 
    i, j in zip(interval_prop_unvacc_vaccinated1, interval_lengths)]

vaccination_rates2 = [
    get_rate_from_coverage_and_duration(i, j) for 
    i, j in zip(interval_prop_unvacc_vaccinated2, interval_lengths)]



In [57]:
pd.Series(
    vaccination_rates1, 
    index=ref_times_to_dti(COVID_BASE_DATE, coverage_times[1:])
).plot(
    kind="scatter",
    title="Rates needed to achieve coverage (partial)",
)

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [58]:
pd.Series(
    vaccination_rates2, 
    index=ref_times_to_dti(COVID_BASE_DATE, coverage_times[1:])
).plot(
    kind="scatter",
    title="Rates needed to achieve coverage (full)",
)

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [59]:
def get_vacc_rate_func(end_times, vaccination_rates1):
    def get_vaccination_rate(time, derived_outputs):

        # Identify the index of the first list element greater than the time of interest
        # If there is such an index, return the corresponding vaccination rate
        for end_i, end_t in enumerate(end_times):
            if end_t > time:
                return vaccination_rates1[end_i]

        # Return zero if the time is after the last end time
        return 0.0
    return get_vaccination_rate

#vacc_rate_func1 = get_vacc_rate_func(coverage_times[1:], vaccination_rates1)
vacc_rate_func1 = get_piecewise_scalar_function(coverage_times[1:], vaccination_rates1)


C:\Users\timot\AppData\Local\Temp\ipykernel_7772\321767980.py:15: DeprecationWarning:

This method is deprecated and scheduled for removal, use get_piecewise_function instead



In [60]:
vacc_rate_func1

Function: 'piecewise_constant', args=(ModelVariable time, Function: '_c...0), kwargs={}), Function: '_c...2), kwargs={})), kwargs={})

In [61]:
def get_vacc_rate_func(end_times, vaccination_rates2):
    def get_vaccination_rate(time, derived_outputs):

        # Identify the index of the first list element greater than the time of interest
        # If there is such an index, return the corresponding vaccination rate
        for end_i, end_t in enumerate(end_times):
            if end_t > time:
                return vaccination_rates2[end_i]

        # Return zero if the time is after the last end time
        return 0.0
    return get_vaccination_rate

#vacc_rate_func2 = get_vacc_rate_func(coverage_times[1:], vaccination_rates2)
vacc_rate_func2 = get_piecewise_scalar_function(coverage_times[1:], vaccination_rates2)

C:\Users\timot\AppData\Local\Temp\ipykernel_7772\3760876679.py:15: DeprecationWarning:

This method is deprecated and scheduled for removal, use get_piecewise_function instead



In [62]:
m.add_transition_flow(
    name='vax_status',
    fractional_rate=vacc_rate_func1,
    source='S',
    dest='S',
    source_strata={'vax_status': 'unvaccinated'},
    dest_strata={'vax_status': 'vaccinated1'},
    # Expected flow count can be used as a sanity check,
    # to assert that the expected number of flows was added.
    #expected_flow_count=1
)


m.add_transition_flow(
    name='vax_status',
    fractional_rate=vacc_rate_func2,
    source='S',
    dest='S',
    source_strata={'vax_status': 'vaccinated1'},
    dest_strata={'vax_status': 'vaccinated2'},
    # Expected flow count can be used as a sanity check,
    # to assert that the expected number of flows was added.
    #expected_flow_count=1
)

In [63]:
# Let's have a look at the matrix using plotly express
print(age_mixing_matrix.shape)
px.imshow(age_mixing_matrix)

(16, 16)


In [64]:
m.flows

[<InfectionFrequencyFlow 'infection' from SXage_0Xvax_status_unvaccinated to EXage_0Xvax_status_unvaccinated>,
 <InfectionFrequencyFlow 'infection' from SXage_0Xvax_status_vaccinated1 to EXage_0Xvax_status_vaccinated1>,
 <InfectionFrequencyFlow 'infection' from SXage_0Xvax_status_vaccinated2 to EXage_0Xvax_status_vaccinated2>,
 <InfectionFrequencyFlow 'infection' from SXage_5Xvax_status_unvaccinated to EXage_5Xvax_status_unvaccinated>,
 <InfectionFrequencyFlow 'infection' from SXage_5Xvax_status_vaccinated1 to EXage_5Xvax_status_vaccinated1>,
 <InfectionFrequencyFlow 'infection' from SXage_5Xvax_status_vaccinated2 to EXage_5Xvax_status_vaccinated2>,
 <InfectionFrequencyFlow 'infection' from SXage_10Xvax_status_unvaccinated to EXage_10Xvax_status_unvaccinated>,
 <InfectionFrequencyFlow 'infection' from SXage_10Xvax_status_vaccinated1 to EXage_10Xvax_status_vaccinated1>,
 <InfectionFrequencyFlow 'infection' from SXage_10Xvax_status_vaccinated2 to EXage_10Xvax_status_vaccinated2>,
 <Infec

In [65]:
parameters = { "rate": 0.3, "detection_rate": 0.1, "symp_prop": 4.5, "asymp_det": 0.2, "vax_effectiveness1": 0.3, "vax_effectiveness2": 0.7}

In [66]:
m.run(parameters)

In [67]:
fig = px.line(m.get_outputs_df())
fig.show()

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [68]:
#m.get_outputs_df().plot()

In [69]:
m.get_derived_outputs_df().plot()

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [70]:
df=pd.read_excel(io='NCRdata_2021.xlsx', index_col=0)  
        #index_col=0 indicates the first column is used as the index of the data (typically date)
notification_data = df["7-DAY MA"]
#death_data = df["CUMULATIVE DEATHS"]

In [71]:
df

,CUMULATIVE,INFECTIONS,7-DAY MA,ACTIVE CASES,Unnamed: 5,Unnamed: 6,Unnamed: 7
DATE,,,,,,,
2021-07-01,527667.0,690.625,686.500083,7263.0,NaN,650,656.857143
2021-07-02,528330.0,663.000,680.229312,7642.0,NaN,624,650.857143
2021-07-03,529032.0,702.000,654.399709,7568.0,NaN,685,626.142857
2021-07-04,529821.0,789.000,654.698317,7677.0,NaN,755,626.428571
2021-07-05,530464.0,643.000,650.965716,7540.0,NaN,615,622.857143
...,...,...,...,...,...,...,...
NaT,NaN,NaN,NaN,NaN,NaN,119,119.571429
NaT,NaN,NaN,NaN,NaN,NaN,159,136.571429
NaT,NaN,NaN,NaN,NaN,NaN,551,205.285714


In [72]:
#df1=pd.read_excel(io='NCRdata_2021.xlsx')  

In [73]:
#df1

In [74]:
notification_data

DATE
2021-07-01    686.500083
2021-07-02    680.229312
2021-07-03    654.399709
2021-07-04    654.698317
2021-07-05    650.965716
                 ...    
NaT                  NaN
NaT                  NaN
NaT                  NaN
NaT                  NaN
NaT                  NaN
Name: 7-DAY MA, Length: 184, dtype: float64

In [75]:
notification_data_cal = notification_data[0:50]
#death_data_cal = death_data[31:70]

In [76]:
notification_data_cal.plot()
#death_data_cal.plot(style='.')

#COMMENT: notification_data_cal.plot() used to be notification_data_cal.plot(style='.'). The latter used to work before without any changes in other parts of the code

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



**Calibration**

In [77]:
# Targets represent data we are trying to fit to
from estival import targets as est

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

In [78]:
targets = [
    est.NormalTarget("notification", notification_data_cal, np.std(notification_data_cal) * 0.1)
    #est.TruncatedNormalTarget("notification", notification_data_cal, (0.0,np.inf),
    #    esp.UniformPrior("notification_dispersion",(0.1, notification_data_cal.max()*0.1))),
    #est.NormalTarget("deaths_cumulative", death_data_cal, np.std(death_data_cal) * 0.1)
]

In [79]:
priors = [
    #esp.TruncNormalPrior("breakpt1",63.0,2.0,(59.0,67.0)),
    #esp.TruncNormalPrior("breakpt2",79.0,2.0,(75.0,83.0)),
    esp.UniformPrior("rate", (0,1)),
   # esp.UniformPrior("multiplier1", (0,1)),
   # esp.UniformPrior("multiplier2", (0.6,1)),
    esp.UniformPrior("detection_rate", (0,0.5)),
    esp.UniformPrior("symp_prop", (4,8)),
    esp.UniformPrior("asymp_det", (0,1)),
    esp.UniformPrior("vax_effectiveness1", (0.1,0.4)),
    esp.UniformPrior("vax_effectiveness2", (0.5,0.9)),
]

In [80]:
#defp = {"breakpt1": 65, "breakpt2": 75, "rate1": 0.2, "rate2": 0.2, "rate3": 0.2, "detection_rate":0.2, "symp_prop": 5, "asymp_det": 0.2, "death_rate": 0.01, "asymp_death": 0.5}
defp = parameters

In [81]:
#COPY/REMOVE CODE BELOW TO SWITCH CALIBRATION METHOD

In [82]:
bcm = BayesianCompartmentalModel(m, defp, priors, targets)


In [83]:
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class)

In [84]:
# You can also suggest starting points for the optimization (as well as specify an init method for unsuggested points)
# This is the "midpoint" method by default (ie the 0.5 ppf of the prior distribution)
orunner = optimize_model(bcm, opt_class=opt_class, suggested=defp, init_method="midpoint")

In [85]:
for i in range(8):
    # Run the minimizer for a specified 'budget' (ie number of evaluations)
    rec = orunner.minimize(500)
    # Print the loss (objective function value) of the current recommended parameters
    print(rec.loss)

7.725225460512591
7.475383402858504
7.402564342518363
7.402564342518363
7.402564342518363
7.364575666538459
7.354179082115313
7.326610240711481


In [86]:
mle_params = rec.value[1]
mle_params

{'rate': 0.03310706340123506,
 'detection_rate': 0.032064829902066,
 'symp_prop': 4.1366230917663405,
 'asymp_det': 0.05366022221575723,
 'vax_effectiveness1': 0.196823767067946,
 'vax_effectiveness2': 0.5442491734072481}

In [87]:
res = bcm.run(mle_params)

In [88]:
target = "notification"

# You can access the targets from the BCM
bcm.targets[target].data.plot()
res.derived_outputs[target].plot()

#COMMENT: bcm.targets[target].data.plot() used to be bcm.targets[target].data.plot(style='.'). The latter used to work before without any changes in other parts of the code

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\timot\miniconda3\envs\summer2\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [89]:
variable = "notification"
m = res.derived_outputs[variable]
fig = go.Figure()

fig = fig.add_trace(go.Scatter(x = m.keys(), y = m, name = "MLE"))
fig = fig.add_trace(go.Scatter(x = notification_data_cal.keys(), y = notification_data_cal, name = "data_fit"))
fig = fig.add_trace(go.Scatter(x = notification_data[50:101].keys(), y = notification_data[50:101], name = "data_post"))
fig.show()